<a href="https://colab.research.google.com/github/Hadse/special-octo/blob/master/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip3 install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [0]:
BATCH_SIZE = 32
 
#transformerer om til Tensor - slags Numpy-list
transform = transforms.Compose([transforms.ToTensor()])

# laster ned MNIST datasettet som torch har liggende inne. lagrer det i trainset var.  
#Transform = none?
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
# loader dataset, med gitt Batch size.  
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# nå laster vi ned test set, over var det train.
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [0]:
class NnGarmentClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        super(NnGarmentClassifier, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.d1 = nn.Linear(12 * 4 * 4, 128)
        self.d2 = nn.Linear(128, 10)


    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.reshape(-1,12*4*4)
        x = self.d1(x)
        x = F.relu(x)

        logits = self.d2(x)
        out = F.softmax(logits, dim=1)
        
        return out

In [0]:
## test the model with 1 batch
model = NnGarmentClassifier()
for images, labels in trainloader:
    out = model(images)
    break

In [0]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NnGarmentClassifier()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## compute accuracy
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    logit = logit.view(batch_size, -1)
    print(target.data)
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [0]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)

        logits = logits.view(BATCH_SIZE, -1)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i)) 

Streaming output truncated to the last 5000 lines.
        1, 6, 2, 3, 0, 3, 1, 2], device='cuda:0')
tensor([4, 4, 4, 7, 9, 7, 1, 7, 5, 3, 2, 1, 0, 7, 1, 6, 5, 7, 8, 4, 5, 0, 5, 6,
        4, 1, 7, 8, 7, 2, 1, 0], device='cuda:0')
tensor([9, 2, 5, 1, 8, 4, 2, 4, 7, 9, 8, 3, 7, 9, 6, 5, 3, 4, 9, 1, 1, 0, 0, 3,
        2, 6, 5, 8, 6, 2, 8, 6], device='cuda:0')
tensor([0, 6, 8, 7, 7, 1, 1, 9, 8, 0, 4, 5, 6, 8, 9, 2, 5, 3, 9, 3, 6, 7, 2, 1,
        1, 4, 8, 2, 0, 4, 3, 8], device='cuda:0')
tensor([3, 7, 3, 1, 7, 4, 6, 5, 4, 8, 7, 2, 8, 2, 0, 5, 6, 1, 3, 9, 6, 1, 2, 4,
        1, 5, 7, 0, 1, 0, 6, 0], device='cuda:0')
tensor([3, 5, 3, 7, 0, 8, 2, 5, 6, 8, 3, 8, 2, 1, 0, 3, 0, 7, 8, 7, 7, 1, 1, 3,
        3, 9, 2, 4, 1, 4, 7, 4], device='cuda:0')
tensor([7, 4, 4, 2, 1, 1, 2, 3, 4, 9, 6, 1, 3, 3, 8, 1, 5, 1, 9, 1, 5, 6, 8, 6,
        6, 6, 3, 5, 0, 0, 6, 7], device='cuda:0')
tensor([4, 2, 7, 5, 4, 6, 1, 6, 0, 2, 4, 7, 8, 6, 5, 1, 9, 5, 0, 1, 5, 8, 1, 3,
        7, 6, 3, 1, 4, 8, 5, 1], device

In [0]:
test_acc = 0.0
for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += get_accuracy(outputs, labels, labels.shape[0])
        
print('Test Accuracy: %.2f'%( test_acc/i))

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8], device='cuda:0')
tensor([3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1, 2, 2, 4, 4, 5, 8, 2, 2,
        8, 4, 8, 0, 7, 7, 8, 5], device='cuda:0')
tensor([1, 1, 2, 3, 9, 8, 7, 0, 2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2,
        0, 6, 5, 3, 6, 7, 1, 8], device='cuda:0')
tensor([0, 1, 4, 2, 3, 6, 7, 2, 7, 8, 5, 9, 9, 4, 2, 5, 7, 0, 5, 2, 8, 6, 7, 8,
        0, 0, 9, 9, 3, 0, 8, 4], device='cuda:0')
tensor([1, 5, 4, 1, 9, 1, 8, 6, 2, 1, 2, 5, 1, 0, 0, 0, 1, 6, 1, 6, 2, 2, 4, 4,
        1, 4, 5, 0, 4, 7, 9, 3], device='cuda:0')
tensor([7, 2, 3, 9, 0, 9, 4, 7, 4, 2, 0, 5, 2, 1, 2, 1, 3, 0, 9, 1, 0, 9, 3, 6,
        7, 9, 9, 4, 4, 7, 1, 2], device='cuda:0')
tensor([1, 6, 3, 2, 8, 3, 6, 1, 1, 0, 2, 9, 2, 4, 0, 7, 9, 8, 4, 1, 8, 4, 1, 3,
        1, 6, 7, 2, 8, 5, 2, 0], device='cuda:0')
tensor([7, 7, 6, 2, 7, 0, 7, 8, 9, 2, 9, 0, 5, 1, 4, 4, 5, 6, 9, 2, 6, 8, 6, 4,
        2,

In [0]:
y_true = []
y_pred = []

for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    for label in labels.cpu().numpy():
      y_true.append(label)
    for pred in outputs.cpu().detach().numpy():
      


SyntaxError: ignored